## Output parser 

Output parser is responsible for taking the output of a model and transforming it to a more suitable format for downstream tasks. Useful when you are using LLMs to generate structured data, or to normalize output from chat models and LLMs.LangChain has lots of different types of output parsers such as JSON, XML, CSV, OutputFixing, RetryWithError, Pydantic, YAML, PandasDataFrame, Enum, Datatime, Structured.

### Output parsing with Prompt Template

Try to make output parsing without langchain output_parser function.. Give some prompts to the make the structure of the code.

In [3]:
# Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()


# Access the API key from the environment
api_key = os.getenv("GOOGLE_GEN_API")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_key=api_key)
result = llm.invoke("Tell me about Dhaka.")
result

AIMessage(content="## Dhaka: The Vibrant Heart of Bangladesh\n\nDhaka, the capital of Bangladesh, is a city pulsing with life, history, and a unique blend of tradition and modernity. Situated on the banks of the Buriganga River, it's a melting pot of cultures, a hub of commerce, and a testament to the resilience of its people. \n\n**Here's a glimpse into what makes Dhaka special:**\n\n**History & Heritage:**\n* **Ancient Origins:** Founded in the 7th century, Dhaka boasts a rich history influenced by Mughal, British, and Bengali cultures. \n* **Architectural Marvels:** Explore the Lalbagh Fort (17th century), the Star Mosque with its intricate mosaic work, and the pink-hued Ahsan Manzil, once the palace of the Nawabs of Dhaka.\n* **Cultural Heritage:** Immerse yourself in the vibrant arts scene with traditional music, dance forms like Kathak, and literature celebrated at the Ekushey Book Fair.\n\n**Modern Metropolis:**\n* **Bustling City Life:** Dhaka is a bustling metropolis with chao

In [4]:
result.content

"## Dhaka: The Vibrant Heart of Bangladesh\n\nDhaka, the capital of Bangladesh, is a city pulsing with life, history, and a unique blend of tradition and modernity. Situated on the banks of the Buriganga River, it's a melting pot of cultures, a hub of commerce, and a testament to the resilience of its people. \n\n**Here's a glimpse into what makes Dhaka special:**\n\n**History & Heritage:**\n* **Ancient Origins:** Founded in the 7th century, Dhaka boasts a rich history influenced by Mughal, British, and Bengali cultures. \n* **Architectural Marvels:** Explore the Lalbagh Fort (17th century), the Star Mosque with its intricate mosaic work, and the pink-hued Ahsan Manzil, once the palace of the Nawabs of Dhaka.\n* **Cultural Heritage:** Immerse yourself in the vibrant arts scene with traditional music, dance forms like Kathak, and literature celebrated at the Ekushey Book Fair.\n\n**Modern Metropolis:**\n* **Bustling City Life:** Dhaka is a bustling metropolis with chaotic traffic, vibra

In [5]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You will response the output in a CSV format"),
    ("user", "Tell me about {domain} in {city}")
])

chain = prompt_template | llm 

result = chain.invoke({"domain": "tourist places, culture, Facilities & issues", "city": "Dhaka"})
print(result.content)

Place,Type,Description,Culture,Facilities,Issues
Lalbag Fort,Historical & Architectural,A 17th-century Mughal fort with gardens, mosques, and a museum,Reflects Mughal architecture and Islamic influence,Guided tours, museum, gardens, occasional events,Limited accessibility for people with disabilities, needs better maintenance
Ahsan Manzil,Historical & Architectural,Former palace of the Nawabs of Dhaka, now a museum showcasing their lifestyle,Offers insights into the lives of Dhaka's elite during British rule,Museum, guided tours, gardens, needs better preservation efforts,Limited accessibility for people with disabilities, needs better crowd management during peak seasons
Dhakeshwari Temple,Religious & Historical,An important Hindu temple dedicated to Goddess Durga, considered the 'heart' of Dhaka,Represents a prominent place of worship for the Hindu community,Facilities for prayer, festivals, and religious gatherings,Lack of parking space, crowded during festivals
Armenian Church,Hist

In [6]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You will response the output in a JSON format"),
    ("user", "Tell me about {domain} in {city}")
])

chain = prompt_template | llm 

result = chain.invoke({"domain": "tourist places, culture, Facilities & issues", "city": "Dhaka"})
print(result.content)

```json
{
  "tourist_places": [
    {
      "name": "Ahsan Manzil",
      "description": "A historic palace, once home to the Nawab of Dhaka, showcasing Mughal architecture and rich history.",
      "type": "Historical",
      "recommended_duration": "2-3 hours"
    },
    {
      "name": "Lalbagh Fort",
      "description": "An incomplete 17th-century Mughal fort with beautiful gardens, showcasing impressive architecture and offering scenic views.",
      "type": "Historical",
      "recommended_duration": "2-3 hours"
    },
    {
      "name": "Dhakeshwari Temple",
      "description": "An important Hindu temple and one of the oldest in Dhaka, considered a significant religious site.",
      "type": "Religious",
      "recommended_duration": "1-2 hours"
    },
    {
      "name": "Star Mosque",
      "description": "A beautifully decorated mosque known for its intricate mosaic work using Chinese porcelain.",
      "type": "Religious",
      "recommended_duration": "1-2 hours"
    },


### Output Parsing with LangChain

We can use an output parser to help users to specify an arbitrary JSON schema via the prompt, query a model for outputs that conform to that schema, and finally parse that schema as JSON.

The JsonOutputParser is one built-in option for prompting for and then parsing JSON output. While it is similar in functionality to the PydanticOutputParser, it also supports streaming back partial JSON objects.

Pydantic in LangChain is used to define and validate data models, which helps in structuring the data passed between various components in a LangChain application.Pydantic is often used to create structured data schemas, like the one in your example, where you define a Joke model with fields setup and punchline.

In [7]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

query = "Dhaka"

parser = JsonOutputParser()
print(parser.get_format_instructions())

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n tell me something about {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": query})

Return a JSON object.


{'response': "Dhaka, the capital of Bangladesh, is a vibrant megacity known for its rich history, bustling streets, and delicious cuisine. Here are some interesting facts: \n\n* **Nickname:** It's often called the 'City of Mosques' due to the abundance of beautiful mosques, including the historic Baitul Mukarram.\n* **Rickshaw Capital:** Dhaka boasts the highest concentration of rickshaws globally, making it a unique and colorful sight.\n* **Cultural Hub:**  The city is home to numerous museums, art galleries, and theaters, showcasing the rich cultural heritage of Bangladesh.\n* **Street Food Paradise:**  Dhaka offers a mouthwatering array of street food, from spicy fuchka to flavorful biryani.\n* **Buriganga River:** The iconic Buriganga River flows through Dhaka, offering scenic boat rides and glimpses of local life."}

In [8]:
parser.get_format_instructions()

'Return a JSON object.'

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up about places description")
    description: str = Field(description="answer to provide some description of that place")
    tourist_places: str = Field(description="answer to provide the best tourist spots of that city")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Dhaka"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\nTell me something about {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'setup': 'Dhaka is the capital and largest city of Bangladesh. It is known for its rich history, vibrant culture, and delicious food.',
 'description': 'Dhaka is a bustling metropolis situated on the banks of the Buriganga River. It is a city of contrasts, with ancient Mughal architecture standing alongside modern skyscrapers. The city is known for its traffic congestion, but also for its friendly people and lively street life.',
 'tourist_places': 'Some of the best tourist spots in Dhaka include the Lalbagh Fort, the Ahsan Manzil Museum, the Star Mosque, and the Dhakeshwari Temple. You can also take a boat ride on the Buriganga River or explore the narrow alleys of Old Dhaka.'}

In [10]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "question to set up about places description", "title": "Setup", "type": "string"}, "description": {"description": "answer to provide some description of that place", "title": "Description", "type": "string"}, "tourist_places": {"description": "answer to provide the best tourist spots of that city", "title": "Tourist Places", "type": "string"}}, "required": ["setup", "description", "tourist_places"]}\n```'

You can also use the JsonOutputParser without Pydantic. This will prompt the model to return JSON, but doesn't provide specifics about what the schema should be.

### XMLParsing

This guide shows you how to use the XMLOutputParser to prompt models for XML output, then and parse that output into a usable format.

In [11]:
from langchain.output_parsers import XMLOutputParser

actor_query = "Generate the shortened filmography for Tom Hanks."

parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])

# We will add these instructions to the prompt below
parser.get_format_instructions()

prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

output = chain.invoke({"query": actor_query})
print(output)

{'movies': [{'actor': [{'name': 'Tom Hanks'}, {'film': [{'name': "He Knows You're Alone"}, {'genre': 'Horror'}]}, {'film': [{'name': 'Mazes and Monsters'}, {'genre': 'Thriller'}]}, {'film': [{'name': 'Splash'}, {'genre': 'Romantic Comedy'}]}, {'film': [{'name': 'Bachelor Party'}, {'genre': 'Comedy'}]}, {'film': [{'name': 'The Man with One Red Shoe'}, {'genre': 'Comedy'}]}, {'film': [{'name': 'Volunteers'}, {'genre': 'Comedy'}]}, {'film': [{'name': 'The Money Pit'}, {'genre': 'Comedy'}]}, {'film': [{'name': 'Every Time We Say Goodbye'}, {'genre': 'Romance'}]}, {'film': [{'name': 'Nothing in Common'}, {'genre': 'Drama'}]}, {'film': [{'name': 'Big'}, {'genre': 'Fantasy Comedy-Drama'}]}, {'film': [{'name': 'Punchline'}, {'genre': 'Drama'}]}, {'film': [{'name': "The 'Burbs"}, {'genre': 'Comedy'}]}, {'film': [{'name': 'Turner & Hooch'}, {'genre': 'Comedy'}]}, {'film': [{'name': 'Joe Versus the Volcano'}, {'genre': 'Comedy'}]}, {'film': [{'name': 'The Bonfire of the Vanities'}, {'genre': 'Sat

In [12]:
parser.get_format_instructions()

'The output should be formatted as a XML file.\n1. Output should conform to the tags below. \n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema. \n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\n[\'movies\', \'actor\', \'film\', \'name\', \'genre\']\n```'

### Create a custom Output Parser
In some situations you may want to implement a custom parser to structure the model output into a custom format. Here, we will make a simple parse that inverts the case of the output from the model.

For example, if the model outputs: "Meow", the parser will produce "mEOW".

In [13]:
from typing import Iterable
from langchain_core.messages import AIMessage, AIMessageChunk



def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = llm | parse
chain.invoke("hello")

'hELLO! hOW CAN i HELP YOU TODAY? \n'

In [14]:
from typing import Iterable
from langchain_core.messages import AIMessage, AIMessageChunk



def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.upper()


chain = llm | parse
chain.invoke("please tell about dhaka")

"## DHAKA: THE CITY OF MOSQUES, RICKSHAWS, AND MUGHAL HISTORY\n\nDHAKA, THE CAPITAL OF BANGLADESH, IS A VIBRANT AND BUSTLING MEGACITY TEEMING WITH LIFE, HISTORY, AND CULTURE. KNOWN FOR ITS CHAOTIC YET CAPTIVATING ATMOSPHERE, DHAKA OFFERS A UNIQUE BLEND OF OLD-WORLD CHARM AND MODERN ASPIRATIONS. HERE'S A GLIMPSE INTO WHAT MAKES DHAKA SPECIAL:\n\n**HISTORICAL TAPESTRY:**\n\n* **MUGHAL LEGACY:** DHAKA BOASTS A RICH MUGHAL HISTORY, EVIDENT IN ITS STUNNING ARCHITECTURE. EXPLORE THE LALBAGH FORT, A 17TH-CENTURY MUGHAL FORT OVERLOOKING THE BURIGANGA RIVER, OR VISIT THE MAJESTIC STAR MOSQUE, ADORNED WITH INTRICATE MOSAIC WORK. \n* **PINK PALACE:** THE AHSAN MANZIL, A MAGNIFICENT PINK PALACE, STANDS AS A TESTAMENT TO DHAKA'S PAST AS A PROMINENT CENTER OF TRADE DURING BRITISH RULE. \n\n**CULTURAL HUB:**\n\n* **RICKSHAW CAPITAL:** EXPERIENCE THE QUINTESSENTIAL DHAKA EXPERIENCE BY HOPPING ONTO A BRIGHTLY DECORATED RICKSHAW, NAVIGATING THE CITY'S BUSTLING STREETS.\n* **FLAVORS OF BANGLADESH:** INDU